In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torchvision import transforms, models
from torch.utils.data import DataLoader, random_split

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
model = models.resnet18(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
# Set hyperparameters
BATCH_SIZE = 128
LR = 0.01

# Prepare dataset and preprocessing
transform_train = transforms.Compose([
    transforms.Resize(32),                  # Resize for ResNet
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))    # Normalization for grayscale
])

transform_test = transforms.Compose([
    transforms.Resize(32),                  # Resize for ResNet
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))    # Normalization for grayscale
])

# Load Fashion MNIST dataset
trainset = torchvision.datasets.FashionMNIST(root='../data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

testset = torchvision.datasets.FashionMNIST(root='../data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)
# Labels in Fashion MNIST
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot')

In [8]:
# Define the BitLinear layer
class BitLinear(nn.Module):
    def __init__(self, in_features, out_features):
        super(BitLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        # Initialize weights and biases close to zero
        self.weight = nn.Parameter(torch.randn(out_features, in_features) * 0.01)
        self.bias = nn.Parameter(torch.randn(out_features) * 0.01)

    def forward(self, x):
        # make weights binary
        binarized_weight = torch.sign(torch.tanh(self.weight))
        return F.linear(x, binarized_weight, self.bias)

# Define the Residual Block and ResNet
class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )

    def forward(self, x):
        out = self.left(x)
        out = out + self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, ResidualBlock, num_classes=10):
        super(ResNet, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False),  # Adjusted for grayscale images
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        self.layer1 = self.make_layer(ResidualBlock, 64, 2, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride=2)
        self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride=2)
        self.fc = BitLinear(512, num_classes)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

def ResNet18():
    return ResNet(ResidualBlock)

In [11]:
net = ResNet18().to(device)

In [12]:
# Define loss function & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)

In [13]:
# Training loop
EPOCH = 10
for epoch in range(EPOCH):
    print('\nEpoch: %d' % (epoch + 1))
    net.train()
    sum_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        # Forward & backward
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Calculate accuracy and loss
        sum_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        # Print every 500th iteration
        if (i + 1) % 50 == 0:
            print('[epoch:%d, iter:%d] Loss: %.03f | Acc: %.3f%% ' %
                  (epoch + 1, i + 1, sum_loss / (i + 1), 100. * correct / total))

    # Test after each epoch
    print('Testing...')
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print('Test Accuracy: %.3f%%' % (100 * correct / total))

print('Training complete.')



Epoch: 1
[epoch:1, iter:50] Loss: 11.153 | Acc: 51.844% 
[epoch:1, iter:100] Loss: 6.063 | Acc: 61.305% 
[epoch:1, iter:150] Loss: 4.320 | Acc: 65.005% 
[epoch:1, iter:200] Loss: 3.438 | Acc: 67.172% 
[epoch:1, iter:250] Loss: 2.889 | Acc: 69.088% 
[epoch:1, iter:300] Loss: 2.516 | Acc: 70.396% 
[epoch:1, iter:350] Loss: 2.240 | Acc: 71.504% 
[epoch:1, iter:400] Loss: 2.034 | Acc: 72.398% 
[epoch:1, iter:450] Loss: 1.878 | Acc: 72.946% 
Testing...
Test Accuracy: 78.660%

Epoch: 2
[epoch:2, iter:50] Loss: 0.606 | Acc: 78.328% 
[epoch:2, iter:100] Loss: 0.569 | Acc: 79.398% 
[epoch:2, iter:150] Loss: 0.555 | Acc: 80.000% 
[epoch:2, iter:200] Loss: 0.541 | Acc: 80.449% 
[epoch:2, iter:250] Loss: 0.531 | Acc: 80.747% 
[epoch:2, iter:300] Loss: 0.525 | Acc: 80.984% 
[epoch:2, iter:350] Loss: 0.521 | Acc: 81.092% 
[epoch:2, iter:400] Loss: 0.517 | Acc: 81.289% 
[epoch:2, iter:450] Loss: 0.516 | Acc: 81.264% 
Testing...
Test Accuracy: 82.730%

Epoch: 3
[epoch:3, iter:50] Loss: 0.466 | Acc: 8